## Real data pipeline

In [1]:
import mne

ModuleNotFoundError: No module named 'mne'

### Intake data

In [14]:
# This dataset is 2.6 MB on disk
url = "https://physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf?download"
local_data_path = "../../data/"

# Will not download if already present at local_data_path
local_file_path = download_file(url, local_data_path)

../../data/S001R04.edf already exists. Skipping download.


In [15]:
raw = mne.io.read_raw_edf(local_file_path, preload=True)
raw.info

Extracting EDF parameters from /Users/droumis/src/neuro/data/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


<Info | 8 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
 subject_info: 3 items (dict)
>

In [30]:
# preview the channel names, types, signal ranges, and uncompressed size
raw.describe()

NameError: name 'raw' is not defined

### Gather the real timeseries annotations and clean up

In [17]:
# get initial time of experiment
orig_time = raw.annotations.orig_time

# get annotations into pd df
annotations_df = raw.annotations.to_data_frame()

# Ensure the 'onset' column is in UTC timezone
annotations_df['onset'] = annotations_df['onset'].dt.tz_localize('UTC')

annotations_df['start'] = (annotations_df['onset'] - orig_time).dt.total_seconds()
annotations_df['end'] = annotations_df['start'] + annotations_df['duration']


unique_descriptions = annotations_df['description'].unique()
color_map = dict(zip(unique_descriptions, cc.glasbey[:len(unique_descriptions)]))
annotations_df['color'] = annotations_df['description'].map(color_map)

annotations_df.head()


,onset,duration,description,start,end,color
0,2009-08-12 16:15:00+00:00,4.2,T0,0.0,4.2,#d60000
1,2009-08-12 16:15:04.200000+00:00,4.1,T2,4.2,8.3,#8c3bff
2,2009-08-12 16:15:08.300000+00:00,4.2,T0,8.3,12.5,#d60000
3,2009-08-12 16:15:12.500000+00:00,4.1,T1,12.5,16.6,#018700
4,2009-08-12 16:15:16.600000+00:00,4.2,T0,16.6,20.8,#d60000


### Clean channel names, set sensor positions, and reference data

In [18]:
# clean up the channel names
raw.rename_channels(lambda s: s.strip("."));

In [19]:
# optional: preview available montages that are shipped with MNE
# mne.channels.get_builtin_montages(descriptions=True)

In [20]:
# optional: Let's use the standard 10-20
# montage = mne.channels.make_standard_montage("standard_1020")

In [21]:
# optional: plot the assigned positions of our data channels
# raw.set_montage(montage, match_case=False)
# sphere=(0, 0.015, 0, 0.099) #manually adjust the y origin coord and radius a bit
# raw.plot_sensors(show_names=True, sphere=sphere);

In [22]:
# re-reference EEG data to the average over all recording channels
raw.set_eeg_reference("average");

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


### Gather the data for plotting into simple numpy arrays

In [23]:
time = raw.times
channels = raw.ch_names

# get the EEG data (for this data set, all channels are EEG anyways)
eeg_indices = mne.pick_types(raw.info, eeg=True)
data = raw.get_data(picks=eeg_indices, units={"eeg":"uV"})

### Visualize real data. Approach: Offset, HoloViz

In [ ]:
max_ch_disp = 20  # max channels to initially display
max_t_disp = 20 # max time in seconds to initially display

spacing = 2.5  # Spacing between channels
offset = np.std(data) * spacing

# Create an overlay of VSpan annotations based on the annotations dataframe
annotation_elements = [hv.VSpan(row['start'], row['end']).opts(fill_color=row['color'], alpha=0.1) 
                       for _, row in annotations_df.iterrows()]
annotations_overlay = hv.Overlay(annotation_elements)

# Create a hv.Curve element per chan
channel_curves = []
max_data = data.max()
 
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "@original_amplitude µV")])

xwheel = WheelZoomTool(
    zoom_together="none",
    dimensions="width",
    maintain_focus=False,
)

for i, channel_data in enumerate(data):
    offset_data = channel_data + (i * offset)
    max_data = max(offset_data.max(), max_data) # update max
    ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
    channel_curves.append(
        hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
            color="black", line_width=1,
            tools=[hover, xwheel], shared_axes=False))

yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

def set_maintain_focus(plot, element):
    wheel_zoom = plot.state.select(type=WheelZoomTool)
    if wheel_zoom:
        wheel_zoom[0].maintain_focus = False

# Create an overlay of curves
eeg_viewer = (annotations_overlay * hv.Overlay(channel_curves, kdims="Channel"))
eeg_viewer = eeg_viewer.opts(
    padding=0, xlabel="Time (s)", ylabel="Channel",
    yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, xlim=(time.min(), time.max()), backend_opts={
        # "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), max_data)},
    hooks=[set_maintain_focus])

# Get the y positions of the yticks to use as yaxis of minimap image
y_positions, _ = zip(*yticks)

# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tiack positions from the eeg_viewer
minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))

# Style the minimap 
clim_mul = 1.2
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))
    
# Create RangeToolLink between the minimap and the main EEG viewer 
max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

# Panel is not working
# eeg_app = pn.Column((eeg_viewer + minimap * annotations_overlay).cols(1), min_height=650)
eeg_app = (eeg_viewer + minimap * annotations_overlay).cols(1)
eeg_app

### Visualize real data. Approach: Subcoord, HoloViz

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Requires HoloViews>=1.18</p>
</div>


In [ ]:
max_ch_disp = 20  # max channels to initially display
max_t_disp = 20 # max time in seconds to initially display

# Create an overlay of VSpan annotations based on the annotations dataframe
annotation_elements = [hv.VSpan(row['start'], row['end']).opts(fill_color=row['color'], alpha=0.1) 
                       for _, row in annotations_df.iterrows()]
annotations_overlay = hv.Overlay(annotation_elements)

channel_curves = []

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

for channel, channel_data in zip(channels, data):
    ds = Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, tools=[hover])
    channel_curves.append(curve)

eeg_viewer = (hv.Overlay(channel_curves, kdims="Channel"))
eeg_viewer = eeg_viewer.opts(
    padding=0, xlabel="Time (s)", ylabel="Channel",
    show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, xlim=(time.min(), time.max()), backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (0, len(channels))})


y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

# Does not currently work with rasterize!!!
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")

clim_mul = 1.2
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# Create RangeToolLink between the minimap and the main EEG viewer
RangeToolLink(minimap, eeg_viewer, axes=["x", "y"], boundsx=(None, 2), boundsy=(None, 6.5))

eeg_app = (eeg_viewer + minimap).opts(merge_tools=False).cols(1)

# Panel is not working
# eeg_app = pn.Column((eeg_viewer + minimap * annotations_overlay).cols(1), min_height=650)

eeg_app